<a href="https://colab.research.google.com/github/NoahBjongHoKim/IML_Projects_BoK/blob/master/template_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


First, we import necessary libraries:

just added whatever was needed, probably also stuff not needed anymore

In [ ]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
import torchvision
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch.optim as optim
# Add any other imports you need here

In [ ]:
# The device is automatically set to GPU if available, otherwise CPU
# If you want to force the device to CPU, you can change the line to
# device = torch.device("cpu")
# When using the GPU, it is important that your model and all data are on the
# same device.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

This class to get an embedding from the pretrained network and not a prediction

In [ ]:
class NoFinalLayer(nn.Module):
    def __init__(self, original_model):
        super(NoFinalLayer, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        return x

Reading all the stupid images and applying the pretrained network on them. Takes ages. Try to avoid.

In [ ]:
def generate_embeddings():
    """
    Transform, resize and normalize the images and then use a pretrained model to extract
    the embeddings.
    """
    # TODO: define a transform to pre-process the images
    # The required pre-processing depends on the pre-trained model you choose
    # below.
    # See https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models

    #Average image width: 453.9186
    #Average image height: 306.4356
    image_size = (224,224)
    #mean per channel =  [tensor(0.6082), tensor(0.5158), tensor(0.4114)]
    #std per channel =  [tensor(0.2222), tensor(0.2382), tensor(0.2561)]
    mean_per_channel = (0.6082, 0.5158, 0.4114)
    standard_deviation_per_channel_reciprocal = (0.2222, 0.2382, 0.2561)

    train_transforms = transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize(mean = mean_per_channel, std= standard_deviation_per_channel_reciprocal),
                             ])

    path = "/content/drive/MyDrive/task3_full"

    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/task3_full/dataset", transform=train_transforms)
    # Hint: adjust batch_size and num_workers to your PC configuration, so that you don't
    # run out of memory (VRAM if on GPU, RAM if on CPU)

    #we have to pass images of the same size to the DataLoader or it is unhappy (just like me)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=64, #teiler von 10001 #maybe set to 32 if it does not work
                              shuffle=False, #set this to true?
                              pin_memory=True, num_workers=16, #and this to 8, google says 2 originally 16
                              )

    # TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
    #  more info here: https://pytorch.org/vision/stable/models.html)
    weights = ResNet50_Weights.DEFAULT
    model_init = resnet50(weights=weights)
    model = NoFinalLayer(model_init)
    model.to(device)
    embedding_size = 2048 #this works, I tried it

    # TODO: pick your model
    num_images = len(train_dataset)
    print(num_images)
    print(embedding_size)
    embeddings_list = []
    # TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
    # model to access the embeddings the model generates.

    model.eval()  # Set the model to evaluation mode
    number=0
    with torch.no_grad():  # Disable gradient computation
        for images, _ in train_loader:
            #print("size :", images.size(), " type:", images.type())
            images = images.to(device)  # Move images to the device
            # Forward pass through the model to get the output
            number = number + 1
            outputs = model(images) # Get both output and auxiliary output
            # Extract embeddings from the output
            embeddings_list.append(outputs.cpu().numpy().squeeze())   # Append embeddings to the list
            print("Batch: ",number, "/",num_images/91)
    embeddings_array = np.concatenate(embeddings_list, axis=0)
    #embeddings_list = np.array(embeddings_list)
    print(embeddings_array.shape)
    assert(np.shape(embeddings_array) == (num_images, embedding_size))
    np.save("/content/drive/MyDrive/task3_full/dataset/embeddings.npy", embeddings_array)

In [ ]:
print(device)
generate_embeddings()


cuda:0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 191MB/s]


10023
2048


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/task3_full/dataset",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/task3_full/dataset/embeddings.npy')
    # TODO: Normalize the embeddings
    #embeddings = torch.nn.functional.normalize(embeddings)

    #Mean per channel: [161.6640330815327, 140.88972730355985, 111.99007629992748]
    #Standard deviation per channel: [1/57.38794298481981, 1/62.1605772577997, 1/70.41062360670296]
    #embeddings = torchvision.transforms.Normalize(embeddings, mean = [161.6640330815327, 140.88972730355985, 111.99007629992748],
    #                                                         std = [57.38794298481981, 62.1605772577997, 70.41062360670296])
    # Compute L2 normalization for embeddings
    #print(embeddings)
    norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
    embeddings_normalized = embeddings / norm
    print(embeddings_normalized)
    # Compute mean vector of the embeddings
    mean_embedding = np.mean(embeddings_normalized, axis=0)

    # Subtract the mean vector from each normalized embedding
    embeddings_normalized_zero_mean = embeddings_normalized - mean_embedding
    print("Mean:", np.mean(embeddings_normalized_zero_mean, axis=0))
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings_normalized_zero_mean[i] #embeddings_normalized_zero_mean[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

In [ ]:
def get_data_triplet(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/task3_full/dataset",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/task3_full/dataset/embeddings.npy')
    # TODO: Normalize the embeddings
    #embeddings = torch.nn.functional.normalize(embeddings)

    #Mean per channel: [161.6640330815327, 140.88972730355985, 111.99007629992748]
    #Standard deviation per channel: [1/57.38794298481981, 1/62.1605772577997, 1/70.41062360670296]
    #embeddings = torchvision.transforms.Normalize(embeddings, mean = [161.6640330815327, 140.88972730355985, 111.99007629992748],
    #                                                         std = [57.38794298481981, 62.1605772577997, 70.41062360670296])
    # Use numpy's L2 normalization
    norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
    # Normalize embeddings
    embeddings_normalized = embeddings / norm
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings_normalized[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        #if train:
        #    X.append(np.hstack([emb[0], emb[2], emb[1]]))
        #    y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

We are given A, B, C
A is the Anchor
B is positive
C is negative
Then y = 1

If we swap B, C (so A, C, B if original)
A is the Anchor
C is negative
B is positive
Then y = 0

So given a triplet P1, P2, P3, label y+1 denotes the index of the negative (in embeddings)


Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory (VRAM if on GPU, RAM if on CPU)

In [ ]:
def create_loader_from_np(X, y = None, train = True, batch_size=64, shuffle=True, num_workers = 4):
    """
    Create a torch.utils.data.DataLoader object from numpy arrays containing the data.

    input: X: numpy array, the features
           y: numpy array, the labels

    output: loader: torch.data.util.DataLoader, the object containing the data
    """
    if train:
        # Attention: If you get type errors you can modify the type of the
        # labels here
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                torch.from_numpy(y).type(torch.long))
    else:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader

TODO: define a model. Here, the basic structure is defined, but you need to fill in the details

In [ ]:
import torch.nn.init as init
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.fc1 = nn.Linear(3*2048, 2048)
        self.fc2 = nn.Linear(2048, 128)
        self.fc3 = nn.Linear(128, 1)

        self.dropout = nn.Dropout(p=0.7)

        # Print initial weights
        print("Initial weights of the linear layer:")
        print(self.fc1.weight)
        print("Initial bias of the linear layer:")
        print(self.fc1.bias)

        # Initialize weights using Xavier initialization
        init.xavier_uniform_(self.fc1.weight)
        init.constant_(self.fc1.bias, 0)  # Initialize bias to zeros

        # Print weights after initialization
        print("Weights after Xavier initialization:")
        print(self.fc1.weight)
        print("Bias after initialization:")
        print(self.fc1.bias)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc3(x))
        return x


In [ ]:
TRAIN_TRIPLETS = '/content/drive/MyDrive/task3_full/train_triplets.txt'

# load the training data
X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
#print(X)
#print(y)
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
# delete the loaded training data to save memory, as the data loader copies
del X
del y

In [ ]:
TEST_TRIPLETS = '/content/drive/MyDrive/task3_full/test_triplets.txt'

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
del X_test
del y_test

In [ ]:
def train_model(data_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data

    output: model: torch.nn.Module, the trained model
    """
    model = Net()
    model.train()
    model.to(device)
    n_epochs = 10
    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part
    # of the training data as a validation split. After each epoch, compute the loss on the
    # validation split and print it out. This enables you to see how your model is performing
    # on the validation data before submitting the results on the server. After choosing the
    # best model, train it on the whole training data.

    criterion = nn.BCELoss()  # Define your loss function here
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)  # Define your optimizer
    #best_model =  Net()


    for epoch in range(n_epochs):
        running_loss = 0.0
        acc= 0.0
        count= 0.0
        #define validation subset
        #validation =


        for i, data in enumerate(train_loader, 0):
            #print("i: ", i, " Data: ", data, "\n")

              inputs, labels = data[0].to(device), data[1].to(device)
              labels = labels.type(torch.float)
              #print("Inputs:", inputs, "Labels:", labels, "\n")
              optimizer.zero_grad()
              outputs = model(inputs)
              #print("Outputs:", outputs.squeeze(), "Labels:", labels, "\n")
              loss = criterion(outputs.squeeze(), labels)

              loss.backward()
              optimizer.step()

              outputs[outputs >= 0.5] = 1
              outputs[outputs < 0.5] = 0
              predicted = outputs

              # Compute accuracy
              acc += (predicted == labels).sum().item()/64
              count += len(labels)

              running_loss += loss.item()
              if i % 200 == 199:  # Print every 20 mini-batches
                  #for name, param in model.named_parameters():
                  #  print(name, param)
                  print(predicted.sum())
                  accper = acc/count
                  print('[%d, %5d] loss: %.3f acc: %.3f' %
                        (epoch + 1, i + 1, running_loss / 200, accper))
                  running_loss = 0.0
              # Get predicted classes as the entry with the highest probability

          #score_validation = validation_acc(validation, model(validation))
          #print("Epoch :" epoch, "Score: ", score_validation, "\n")


    print('Finished Training')
    return model

In [ ]:
model=train_model(train_loader)

In [ ]:
def test_model(model, loader):
    """
    The testing procedure of the model; it accepts the testing data and the trained model and
    then tests the model on it.

    input: model: torch.nn.Module, the trained model
           loader: torch.data.util.DataLoader, the object containing the testing data

    output: None, the function saves the predictions to a results.txt file
    """
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            #predicted = [i >= 0.5 for i in predicted]
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("/content/drive/MyDrive/task3_full/results.txt", predictions, fmt='%i')
    return predictions

In [ ]:
test_model(model, test_loader)